In [1]:
%%capture
from numpy import int16
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from components.ovc_actif import actif_in_Q1Q2, actif_in_Q1strict, actif_in_Q2strict, actif_in_Q1, actif_in_Q2
from pandas import concat

from datetime import datetime
date_du_jour = datetime.today().strftime("%d_%m_%Y")

In [2]:
actif_in_Q1.id_patient.count()
actif_in_Q2.id_patient.count()

if not ((actif_in_Q1.id_patient.count() == actif_in_Q1Q2.id_patient.count() + actif_in_Q1strict.id_patient.count()) and (actif_in_Q2.id_patient.count() == actif_in_Q1Q2.id_patient.count() + actif_in_Q2strict.id_patient.count())):
    raise ValueError("Qi <> Qistrict + QiQj, bad logic")
else:
    print("the logic  Qi = Qistrict + QiQj is valid")

8572

5607

the logic  Qi = Qistrict + QiQj is valid


In [3]:
actif_in_Q2Q1 = actif_in_Q2[actif_in_Q2.id_patient.isin(actif_in_Q1.id_patient)]
actif_in_Q2Q1.id_patient.count()

#### check the symetrie

actif_in_Q1Q2 = actif_in_Q1[actif_in_Q1.id_patient.isin(actif_in_Q2.id_patient)]
actif_in_Q1Q2.id_patient.count()

1004

1004

In [4]:
actif_in_Q2strict.id_patient.count()
actif_in_Q1strict.id_patient.count()
actif_in_Q1Q2.id_patient.count()

print("Find Parenting:")
actif_in_Q2strict.parenting_detailed.unique()
actif_in_Q1strict.parenting_detailed.unique()
actif_in_Q1Q2.parenting_detailed.unique()

4603

7568

1004

Find Parenting:


array(['no', 'yes', 'has_started'], dtype=object)

array(['no', 'has_started', 'yes'], dtype=object)

array(['no', 'has_started'], dtype=object)

In [5]:
completed_Q2strict =  actif_in_Q2strict[actif_in_Q2strict.complete_at_least=="yes"]
completed_Q1strict =  actif_in_Q1strict[actif_in_Q1strict.complete_at_least=="yes"]
completed_Q1Q2 =  actif_in_Q1Q2[actif_in_Q1Q2.complete_at_least=="yes"]

completed_Q2strict.id_patient.count()
completed_Q1strict.id_patient.count()
completed_Q1Q2.id_patient.count()

print('Find Parenting:')
completed_Q2strict.parenting_detailed.unique()
completed_Q1strict.parenting_detailed.unique()
completed_Q1Q2.parenting_detailed.unique()

4528

6543

994

Find Parenting:


array(['no', 'yes', 'has_started'], dtype=object)

array(['no', 'has_started', 'yes'], dtype=object)

array(['no', 'has_started'], dtype=object)

# Enrolled Q2

In [6]:
enrolledQ2_CQ2strict = completed_Q2strict[completed_Q2strict.isEnrolledQ2=="yes"]
enrolledQ2_CQ1strict = completed_Q1strict[completed_Q1strict.isEnrolledQ2=="yes"]
enrolledQ2_CQ1Q2 = completed_Q1Q2[completed_Q1Q2.isEnrolledQ2=="yes"]


enrolledQ2_CQ2strict.id_patient.count()
enrolledQ2_CQ1strict.id_patient.count()
enrolledQ2_CQ1Q2.id_patient.count()


""" 

enrolledQ2_CQ1Q2[enrolledQ2_CQ1Q2.id_patient.isin(enrolledQ2_CQ2strict.id_patient)].id_patient.count()
enrolledQ2_CQ1Q2[enrolledQ2_CQ1Q2.id_patient.isin(enrolledQ2_CQ1strict.id_patient)].id_patient.count()
enrolledQ2_CQ2strict[enrolledQ2_CQ2strict.id_patient.isin(enrolledQ2_CQ1strict.id_patient)].id_patient.count()


enrolledQ2_CQ2strict[~enrolledQ2_CQ2strict.id_patient.isin(enrolledQ2_CQ1strict.id_patient)].id_patient.count()
enrolledQ2_CQ1Q2[~enrolledQ2_CQ1Q2.id_patient.isin(enrolledQ2_CQ1strict.id_patient)].id_patient.count()
enrolledQ2_CQ1Q2[~enrolledQ2_CQ1Q2.id_patient.isin(enrolledQ2_CQ2strict.id_patient)].id_patient.count() 
"""


print('find parenting')
enrolledQ2_CQ2strict.parenting_detailed.unique()

3071

125

0

' \n\nenrolledQ2_CQ1Q2[enrolledQ2_CQ1Q2.id_patient.isin(enrolledQ2_CQ2strict.id_patient)].id_patient.count()\nenrolledQ2_CQ1Q2[enrolledQ2_CQ1Q2.id_patient.isin(enrolledQ2_CQ1strict.id_patient)].id_patient.count()\nenrolledQ2_CQ2strict[enrolledQ2_CQ2strict.id_patient.isin(enrolledQ2_CQ1strict.id_patient)].id_patient.count()\n\n\nenrolledQ2_CQ2strict[~enrolledQ2_CQ2strict.id_patient.isin(enrolledQ2_CQ1strict.id_patient)].id_patient.count()\nenrolledQ2_CQ1Q2[~enrolledQ2_CQ1Q2.id_patient.isin(enrolledQ2_CQ1strict.id_patient)].id_patient.count()\nenrolledQ2_CQ1Q2[~enrolledQ2_CQ1Q2.id_patient.isin(enrolledQ2_CQ2strict.id_patient)].id_patient.count() \n'

find parenting


array(['no'], dtype=object)

# Concatenation

In [7]:
all_ovc = concat([completed_Q1Q2,enrolledQ2_CQ2strict])

In [8]:
all_ovc.drop_duplicates("id_patient",inplace=True)
all_ovc.id_patient.count()
all_ovc.test_results.fillna("0,",inplace=True)
all_ovc.test_results.value_counts()

4065

0,       3714
2,        180
0,,2,     166
0,,3,       2
2,,3,       1
3,          1
1,          1
Name: test_results, dtype: int64

In [9]:
ovc_data = all_ovc[
    (all_ovc.test_results!="0,,3,")&
    (all_ovc.test_results!="0,,1,,2,")&
    (all_ovc.test_results!="0,,2,,3,")&
    (all_ovc.test_results!="3,")&
    (all_ovc.test_results!="2,,3,")
]
ovc_data.id_patient.count()
ovc_data.test_results.value_counts()

4061

0,       3714
2,        180
0,,2,     166
1,          1
Name: test_results, dtype: int64

# postive OVC_DREAMS

In [10]:
positive_dreams= ovc_data[
    (ovc_data.test_results=="1,")|
    (ovc_data.test_results=="0,,1,")
]
positive_dreams.id_patient.count()

1

In [11]:
positive_dreams

,id_patient,id_parenting_group,departement,commune,nbre_pres_for_inter,nbre_parenting_coupe_present,has_comdom_topic,has_preventive_vbg,number_of_condoms_sensibilize,number_condoms_sensibilization_date_in_the_interval,...,ps_1519,ps_2024,secondary_1014,secondary_1519,secondary_2024,complete_1014,complete_1519,complete_2024,complete_at_least,isEnrolledQ2
855,151637,NaN,Artibonite,Grande Saline,21,0,yes,yes,1,1,...,no,primary,no,no,secondary,no,no,no,yes,yes


# OVC_SERV_DREAMS

In [12]:
ovc_data.ovc_age.unique()
ovc_data.parenting.unique()
ovc_data.parenting_detailed.unique()

array(['15-17', '18-24', '10-14', '25-29'], dtype=object)

array(['no'], dtype=object)

array(['no', 'has_started'], dtype=object)

In [13]:
ovcS_dreams = ovc_data[
    (ovc_data.ovc_age=="10-14")|
    (ovc_data.ovc_age=="15-17")
]
ovcS_dreams.id_patient.count()
ovcS_dreams['Gender'] = "Female"

2894

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_5938/4236012469.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ovcS_dreams['Gender'] = "Female"


In [14]:
ovcS_dreams.parenting_detailed.unique()

array(['no', 'has_started'], dtype=object)

# Format Datim

In [15]:
datim_ovc_dreams = ovcS_dreams.pivot_table(index="commune",values="id_patient",columns=["Gender","ovc_age"],aggfunc='count',fill_value=0)

In [16]:
datim_ovc_dreams[('Female','<1')] = 0
datim_ovc_dreams[('Female','1-4')] = 0
datim_ovc_dreams[('Female','5-9')] = 0
datim_ovc_dreams[('Female','18+')] = 0
datim_ovc_dreams[('Male','<1')] = 0
datim_ovc_dreams[('Male','1-4')] = 0
datim_ovc_dreams[('Male','5-9')] = 0
datim_ovc_dreams[('Male','10-14')] = 0
datim_ovc_dreams[('Male','15-17')] = 0
datim_ovc_dreams[('Male','18+')] = 0

columns_orientation =[
    ('Female','<1'),
    ('Female','1-4'),
    ('Female','5-9'),
    ('Female','10-14'),
    ('Female','15-17'),   
    ('Female','18+'),  
    ('Male','<1'),
    ('Male','1-4'),
    ('Male','5-9'),
    ('Male','10-14'),
    ('Male','15-17'),   
    ('Male','18+'),   
]

datim_ovc_dreams = datim_ovc_dreams.reindex(columns= columns_orientation)
datim_ovc_dreams

Gender                         Female                         Male          \
ovc_age                            <1 1-4 5-9 10-14 15-17 18+   <1 1-4 5-9   
commune                                                                      
Cap-Haïtien                         0   0   0    80    44   0    0   0   0   
Carrefour                           0   0   0    81    68   0    0   0   0   
Delmas                              0   0   0     8    21   0    0   0   0   
Desdunes                            0   0   0    63    35   0    0   0   0   
Dessalines                          0   0   0   240    45   0    0   0   0   
Grande Saline                       0   0   0    98    36   0    0   0   0   
Gressier                            0   0   0    57    32   0    0   0   0   
Kenscoff                            0   0   0   540   158   0    0   0   0   
La Chapelle                         0   0   0    45    17   0    0   0   0   
Liancourt                           0   0   0    43    45   0    0   0   0   
Limonade                            0   0   0     0     1   0    0   0   0   
Milot                               0   0   0    16    11   0    0   0   0   
Montrouis                           0   0   0    51    40   0    0   0   0   
Petite Rivière de l'Artibonite      0   0   0    95    51   0    0   0   0   
Plaine-du-Nord                      0   0   0    26     5   0    0   0   0   
Port-au-Prince                      0   0   0    56    47   0    0   0   0   
Pétionville                         0   0   0    47    54   0    0   0   0   
Quartier-Morin                      0   0   0    17     4   0    0   0   0   
Saint-Marc                          0   0   0   134   100   0    0   0   0   
Tabarre                             0   0   0    58    41   0    0   0   0   
Verrettes                           0   0   0   182   102   0    0   0   0   

Gender                                          
ovc_age                        10-14 15-17 18+  
commune                                         
Cap-Haïtien                        0     0   0  
Carrefour                          0     0   0  
Delmas                             0     0   0  
Desdunes                           0     0   0  
Dessalines                         0     0   0  
Grande Saline                      0     0   0  
Gressier                           0     0   0  
Kenscoff                           0     0   0  
La Chapelle                        0     0   0  
Liancourt                          0     0   0  
Limonade                           0     0   0  
Milot                              0     0   0  
Montrouis                          0     0   0  
Petite Rivière de l'Artibonite     0     0   0  
Plaine-du-Nord                     0     0   0  
Port-au-Prince                     0     0   0  
Pétionville                        0     0   0  
Quartier-Morin                     0     0   0  
Saint-Marc                         0     0   0  
Tabarre                            0     0   0  
Verrettes                          0     0   0

In [17]:
# datim_ovc_dreams.reset_index()

# Save the data

In [18]:
#positive_dreams.to_excel('./ovc_results/positive_data/positive_dreams.xlsx',na_rep="",index=False)
#datim_ovc_dreams.to_excel('./ovc_results/datim_ovc_dreams.xlsx',na_rep="",index=True)

ovcS_dreams.to_excel(f"./ovc_results/valid_ovc_data_{date_du_jour}.xlsx",na_rep="",index=False)
positive_dreams.to_excel(f"./ovc_results/positive_data/positive_dreams_{date_du_jour}.xlsx",na_rep="",index=False)
datim_ovc_dreams.to_excel(f"./ovc_results/datim_ovc_dreams_{date_du_jour}.xlsx",na_rep="",index=True)

